In [1]:
import os
import pandas as pd

In [2]:
def load_fmap_log(folder_path):
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                 if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.csv')]
    aggregate_result = pd.DataFrame()
    for record in csv_files:
        df = pd.read_csv(record)
        aggregate_result = aggregate_result.append(df.groupby(['model', 'rank', 'beta', 'lambda_l2', 'lambda_keyphrase', 'lambda_latent', 'lambda_rating', 'learning_rate', 'epoch', 'corruption', 'optimizer'], as_index=False)
 .agg({'F-MAP@5':'mean', 'F-MAP@10':'mean','F-MAP@20':'mean'})).sort_values(by=['beta'])
    return aggregate_result

In [3]:
df = load_fmap_log(folder_path='tables/CDsVinyl_fmap/')

In [4]:
df

,model,rank,beta,lambda_l2,lambda_keyphrase,lambda_latent,lambda_rating,learning_rate,epoch,corruption,optimizer,F-MAP@5,F-MAP@10,F-MAP@20
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.033780,0.029945,0.028049
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.022160,0.024864,0.025901
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.023313,0.025852,0.024185
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.027107,0.028075,0.029163
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.025660,0.025876,0.026653
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.018500,0.021204,0.021857
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.024967,0.024124,0.025855
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.026067,0.025072,0.026176
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.024107,0.024742,0.024467
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.027053,0.028509,0.029671


In [5]:
(df.loc[(df['F-MAP@5'] != 0) & (df['F-MAP@10'] != 0) & (df['F-MAP@20'] != 0)].groupby(['model', 'rank', 'beta', 'lambda_l2', 'lambda_keyphrase', 'lambda_latent', 'lambda_rating', 'learning_rate', 'epoch', 'corruption', 'optimizer'], as_index=False)
.agg({'F-MAP@5':'mean', 'F-MAP@10':'mean','F-MAP@20':'mean'})).sort_values(by=['beta'])

,model,rank,beta,lambda_l2,lambda_keyphrase,lambda_latent,lambda_rating,learning_rate,epoch,corruption,optimizer,F-MAP@5,F-MAP@10,F-MAP@20
0,E-CDE-VAE,200,0.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.025297,0.026176,0.026567
1,E-CDE-VAE,200,0.0001,0.01,1,1,1,0.0001,600,0.4,Adam,0.026987,0.027489,0.027125
2,E-CDE-VAE,200,0.0010,0.01,1,1,1,0.0001,600,0.4,Adam,0.025912,0.025850,0.026034
3,E-CDE-VAE,200,0.0100,0.01,1,1,1,0.0001,600,0.4,Adam,0.025307,0.025785,0.025915
4,E-CDE-VAE,200,0.1000,0.01,1,1,1,0.0001,600,0.4,Adam,0.024448,0.023525,0.023787
5,E-CDE-VAE,200,1.0000,0.01,1,1,1,0.0001,600,0.4,Adam,0.001084,0.001433,0.001704


# Bar Plot for best beta

In [ ]:
def pandas_bar_plot(df, x, y, hue, x_name, y_name, folder='figures', name='unknown', save=True):
    fig, ax = plt.subplots(figsize=(6, 3))
    sns.barplot(ax=ax, x=x, y=y, hue=hue, data=df, errwidth=1, edgecolor='black', facecolor=(1, 1, 1, 0)) #, errwidth=0.5

    num_category = len(df[x].unique())
    hatch = None
    hatches = itertools.cycle(['//', '**', '////', '----', 'xxxx', '\\\\\\\\', ' ', '\\', '...', 'OOO', "++++++++"])
    for i, bar in enumerate(ax.patches):
        if i % num_category == 0:
            hatch = next(hatches)
        bar.set_hatch(hatch)

    plt.xlabel(x_name)
    plt.ylabel(y_name)
    #plt.xticks(rotation=15)
    plt.legend(loc='upper left', ncol=5)
    # if 'Precision' not in y:
    ax.legend_.remove()
    plt.tight_layout()
    if save:
        fig_path = load_yaml('config/global.yml', key='path')['figs']
        plt.savefig("{2}/{0}/{1}_bar.pdf".format(folder, name, fig_path), format="pdf")
        plt.savefig("{2}/{0}/{1}_bar.png".format(folder, name, fig_path), format="png")

        fig_leg = plt.figure(figsize=(12, 0.7))
        ax_leg = fig_leg.add_subplot(111)
        ax_leg.legend(*ax.get_legend_handles_labels(), loc='center', ncol=10)
        ax_leg.axis('off')
        fig_leg.savefig('figs/bar_legend.pdf', format='pdf')

    else:
        plt.show()
    plt.close()

# Line Plot for different beta